In [55]:
%pip install pypdf


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [56]:
%pwd

'c:\\Users\\ual-laptop'

In [57]:
import os
os.chdir("../")

In [58]:
%pwd

'c:\\Users'

In [59]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings

In [60]:
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf",loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [61]:
extracted_data=load_pdf_file(data='C:/Users/ual-laptop/AI&XRStudio/New/SanoCare-AIHealthAssistant/Data/')

KeyboardInterrupt: 

In [ ]:
#extracted_data

In [62]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)
len(text_chunks)

In [ ]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings





In [64]:
embeddings = download_hugging_face_embeddings()

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [66]:
PINECONE_API_KEY= os.getenv('PINECONE_API_KEY')
GROQ_API_KEY= os.getenv('GROQ_API_KEY')

GROQ_API_KEY

'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'

In [67]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

In [68]:
index_name = "sanocare"

In [69]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [71]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
) 

In [72]:
docsearch

In [73]:
retriever= docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [74]:
retrieved_docs = retriever.invoke("What is angioplasty?")

In [75]:
retrieved_docs

[Document(id='349a573f-0304-43d5-a057-f17490e4c04d', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 241.0, 'page_label': '212', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data\\The-Gale-Encyclopedia-of-Medicine.pdf', 'total_pages': 4505.0}, page_content='mia) or an irregular placement or location of blood\nvessels. The results of an angiography vary widely by\nthe type of procedure performed, and should be inter-\npreted and explained to the patient by a trained\nradiologist.\nResources\nBOOKS\nBaum, Stanley, and Michael J. Pentecost, editors.Abrams’\nAngiography. 4th ed. Philadelphia: Lippincott-Raven,\n1996.\nPaula Anne Ford-Martin\nAngiomas see Birthmarks\nAngioplasty\nDefinition\nAngioplasty is a term describing a procedure used'),
 Document(id='b4ac9980-d0c9-416f-a591-6321f56afb73', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-1

In [76]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

In [77]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [1]:
import requests

# Step 1: List all valid mortality-related indicators
indicators = requests.get("https://ghoapi.azureedge.net/api/Indicator").json()
filtered = [ind for ind in indicators["value"] if "mortality" in ind["IndicatorName"].lower()]
print([ind["IndicatorCode"] for ind in filtered])

['imr', 'MORTADO', 'nmr', 'MDG_0000000001', 'CHILDMORT5TO14', 'MDG_0000000026', 'MDG_0000000032', 'MDG_0000000007', 'SA_0000001472', 'SA_0000001473', 'SDGWSHBOD', 'SDGPOISON', 'SDGROADAGE', 'u5mr', 'WHS2_161', 'WHS2_131', 'WHS2_160', 'WHOSIS_000004', 'WHS10_4', 'WHS10_5', 'WHOSIS_000003', 'GHE_YLLNUM', 'GHE_YLLRATE', 'NCD_CCS_MORT_TARGET', 'WHOSIS_000016', 'PRISON_D3_DEATHS_SUICIDE_MRATE', 'PRISON_D3_DEATHS_COVID_MRATE', 'CHILDMORT10TO19', 'MALARIA_EST_MORTALITY', 'CHILDMORT_MORTALITY_10TO14', 'GHE_YLL_NUMERIC', 'PRISON_D3_DEATHS_DRUG_MRATE']


In [5]:
import os
print("OpenAI Key Present:", "OPENAI_API_KEY" in os.environ)

OpenAI Key Present: True


In [7]:
from pathlib import Path


print("\n2. File scan:")
found = False
for f in Path('.').rglob('*'):
    if f.suffix in ('.py', '.env', '.ipynb'):
        try:
            content = f.read_text()
            if 'openai' in content.lower():
                print(f"⚠️ OpenAI reference found in: {f}")
                found = True
        except:
            continue
print("No OpenAI references found" if not found else "")


2. File scan:
No OpenAI references found


In [10]:
print("\n3. Installed packages:")
!pip list 


3. Installed packages:


Package                   Version
------------------------- -----------
aiohappyeyeballs          2.6.1
aiohttp                   3.10.11
aiosignal                 1.3.2
annotated-types           0.7.0
anyio                     4.9.0
asttokens                 3.0.0
attrs                     25.3.0
blinker                   1.9.0
certifi                   2025.1.31
cffi                      1.17.1
charset-normalizer        3.4.1
click                     8.1.8
colorama                  0.4.6
comm                      0.2.2
dataclasses-json          0.6.7
debugpy                   1.8.13
decorator                 5.2.1
distro                    1.9.0
dotenv-python             0.0.1
executing                 2.2.0
filelock                  3.18.0
Flask                     3.1.0
frozenlist                1.5.0
fsspec                    2025.3.0
greenlet                  3.1.1
groq                      0.19.0
h11                       0.14.0
httpcore                  1.0.7
httpx            

In [79]:
# system_prompt = (
#     "You are an AI Health Assistant designed to provide clear, empathetic, and"
#     " informative responses across four categories: General Health, Symptoms Checker, Lifestyle, and Mental Health."
#     " Offer practical advice on wellness, symptom diagnosis and provide more suitable symptomn condition/disease based on the input, and support for mental well-being."
#     " Detect emergency keywords such as \"chest pain,\" \"difficulty breathing,\" or \"suicidal thoughts,\" and promptly advise users"
#     " to seek immediate medical attention without providing direct medical instructions. Maintain a calm, supportive tone, avoid inducing "
#     "fear, and emphasize that your guidance is informational and should not replace professional medical advice. If a user asks an unrelated"
#     " question, politely redirect the conversation to health-related topics or offer clarification to understand their concern."
#     "Additionally whenever responding to the user make sure to carefully evaluate their level of communication (Are they sharing alot or they seem to be shy?) and then evaluate the response before sending it to them. Think about them their state of mind and then reply appropriately."
#     "Try mirroring your patient...if they short prompts try giving them shorter responses."
#     "\n\n"
#     "{context}"
# )

system_prompt = """You are an AI Health Assistant designed to provide clear, empathetic, and informative responses across four categories: General Health, Symptoms Checker, Lifestyle, and Mental Health." 
Offer practical advice on wellness, symptom diagnosis and provide more suitable symptomn condition/disease based on the input, and support for mental well-being. 
Detect emergency keywords such as \"chest pain,\" \"difficulty breathing,\" or \"suicidal thoughts,\" and promptly advise users 
to seek immediate medical attention without providing direct medical instructions. Maintain a calm, supportive tone, avoid inducing fear, and emphasize that your guidance is informational and should not replace professional medical advice. 
If a user asks an unrelated question, politely redirect the conversation to health-related topics or offer clarification to understand their concern.
Additionally whenever responding to the user make sure to carefully evaluate their level of communication (Are they sharing alot or they seem to be shy?) and then evaluate the response before sending it to them. Think about them their state of mind and then reply appropriately.
Try mirroring your patient...if they short prompts try giving them shorter responses.
\n\n
{context}
"""

In [80]:

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [81]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [82]:
response = rag_chain.invoke({"input": "I have chest pain"})
print(response["answer"])

Chest pain can be concerning. I want to help you, but I need to make sure you're safe. If you're experiencing severe chest pain, difficulty breathing, or other concerning symptoms, please seek immediate medical attention. Can you tell me more about your chest pain? Where is it located, and when did it start?


In [ ]:
import os
print(os.environ.get("GROQ_API_KEY"))  # Should show your key